In [5]:
import pandas as pd

In [14]:
pdf_df = pd.read_excel("pdf_links.xlsx")
pdf_df.shape

(496, 1)

In [7]:
traversed_df = pd.read_excel("traversed_links.xlsx")
traversed_df

,traversed_link
0,https://www.cncbinternational.com/home/en/inde...
1,https://www.cncbinternational.com/personal/inv...
2,https://www.cncbinternational.com/internationa...
3,https://www.cncbinternational.com/personal/e-b...
4,https://www.cncbinternational.com/personal/mor...
...,...
331,https://www.cncbinternational.com/buy_order.html
332,https://www.cncbinternational.com/index
333,https://www.cncbinternational.com/cash-in-inst...
334,https://www.cncbinternational.com/airport_limo...


In [11]:
pd.set_option("display.max_colwidth", None) 
traversed_df[traversed_df["traversed_link"].str.contains("monopoly")]

,traversed_link
13,https://www.cncbinternational.com/personal/e-banking/inmotion/en/offers/monopoly-202109/index.html
294,https://www.cncbinternational.com/monopoly-202109/index.html


In [13]:
pdf_df[pdf_df["pdf_link"].str.contains("fees-and")]

,pdf_link
23,https://www.cncbinternational.com/_document/investment-products-service-fees-and-charges/en/investment-products-service-fees-and-charges.pdf
29,https://www.cncbinternational.com/_document/private-banking/en/private-banking-investment-products-and-bank-service-fees-and-charges-obsoleted-by-18-december-2023.pdf
42,https://www.cncbinternational.com/_document/bank-service-fees-and-charges/en/private-banking-investment-products-bank-service-fee-and-charges-singapore.pdf
95,https://www.cncbinternational.com/_document/bank-service-fees-and-charges/en/bank-service-fees-and-charges.pdf
105,https://www.cncbinternational.com/_document/previous-noc/en/bank-service-fees-and-charges-bb.pdf
147,https://www.cncbinternational.com/_document/private-banking/en/pb-fees-and-charges.pdf
161,https://www.cncbinternational.com/_document/bank-service-fees-and-charges/en/general-services.pdf
178,https://www.cncbinternational.com/_document/previous-noc/en/bank-service-fees-and-charges-obsoleted-240524.pdf
217,https://www.cncbinternational.com/_document/bank-service-fees-and-charges/en/terms-and-conditions-for-citicfirst-wealth-management-service.pdf
226,https://www.cncbinternational.com/_document/bank-service-fees-and-charges/en/wholesale-banking-fees-and-charges.pdf


In [15]:
import requests

def download_file(url, filename):
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Open the file in binary write mode
        with open(filename, 'wb') as file:
            # Iterate over the response data in chunks
            for chunk in response.iter_content(chunk_size=8192):
                # Write each chunk to the file
                file.write(chunk)
        
        print(f"File downloaded successfully: {filename}")
    except requests.RequestException as e:
        print(f"Error downloading file: {e}")

# Example usage
url = "https://www.cncbinternational.com/_document/previous-noc/en/bank-service-fees-and-charges.pdf"
filename = "pdf_files/bank-service-fees-and-charges.pdf"
download_file(url, filename)

File downloaded successfully: bank-service-fees-and-charges.pdf
